# **0. Imports**

In [41]:
import pandas as pd
import numpy as np

## **0.1 Data**

In [59]:
data = pd.read_csv('../data/data_trab2_econometria.csv')

,Data,IPCA_ACUMULADO,EXP_MÉDIA_ACUMULADA_IPCA,CAMBIO,JUROS_NOMINAL
0,2003.0,30.608719,11.346410,3.4376,26.377288
1,2003.0,20.558350,11.236605,3.5900,24.310818
2,2003.0,15.802754,10.598159,3.4461,23.580332
3,2003.0,12.280552,9.483520,3.1179,24.898055
4,2003.0,7.571674,9.137506,2.9549,26.377288
